In [ ]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/all_kindle_review .csv')

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [ ]:
df = df[['reviewText', 'rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [ ]:
df.shape

(12000, 2)

In [ ]:
df.isnull().sum()

,0
reviewText,0
rating,0


In [ ]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [ ]:
df['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


In [ ]:
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

In [ ]:
df['reviewText'] = df['reviewText'].str.lower()

In [ ]:
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-z A-z 0-9]+', '', x))
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join ([y for y in x.split() if y not in stopwords.words('english')]))
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','', str(x)))
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))

In [ ]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read i didnt want put i read one s...,1
2,ill start saying first four books i wasnt expe...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,i expect type book library pleased find price ...,1


In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join([lemmatizer.lemmatize(y) for y in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['rating'], test_size=0.2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000)
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [ ]:
from sklearn.naive_bayes import GaussianNB
bow_model = GaussianNB().fit(X_train_bow, y_train)
tfidf_model = GaussianNB().fit(X_train_tfidf, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
y_pred_bow = bow_model.predict(X_test_bow)
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

In [ ]:
print(accuracy_score(y_test, y_pred_bow))
print(accuracy_score(y_test, y_pred_tfidf))

0.60375
0.76


In [ ]:
pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 36.6 MB/s eta 0:00:00


In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
corpus=[]
for i in range(0, len(df)):
  review = re.sub('[^a-zA-Z]', ' ', df['reviewText'][i])
  print(i)
  review = review.lower()
  review = review.split()
  review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7153
7154
7155
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189

In [ ]:
[[i,j,k] for i,j,k in zip(map(len, corpus), corpus, df['reviewText']) if i<1]

[]

In [ ]:
corpus

['jace rankin may short nothing mess man hauled saloon undertaker know famous bounty hunter oregon shot man saloon finished year long quest avenge sister murder trying figure next snottynosed farm boy rescued gang bully offer money kill man forced ranch reluctantly agrees bring man justice kill outright first need tell sister widower newskyla kyle springer bailey riding trail sleeping ground past month trying find jace want revenge man killed husband took ranch amongst crime shes keen detour jace want take realizes shes option hide behind boy persona best try keep pace confrontation along way get shot jace discovers kyles kyla come clean whole reason need scoundrel dead hope hell still help herthe book share touching moment slowblooming romance kyla find good reason fear men hide behind boy persona watching jace slowly pull shell help conquer fear endearing pain real deeplyrooted didnt disappear face sexiness neither understandable aversion marriage magically disappear round nookie wou

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
words = []
nltk.download('punkt_tab')
for sent in corpus:
  sent_token = sent_tokenize(sent)
  for sent in sent_token:
    words.append(simple_preprocess(sent))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import gensim
model = gensim.models.Word2Vec(words, window=5, min_count=5)

In [ ]:
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'much',
 'first',
 'well',
 'even',
 'didnt',
 'short',
 'know',
 'way',
 'could',
 'great',
 'make',
 'sex',
 'little',
 'dont',
 'thing',
 'want',
 'two',
 'think',
 'find',
 'plot',
 'also',
 'romance',
 'end',
 'life',
 'im',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'take',
 'woman',
 'many',
 'lot',
 'written',
 'kindle',
 'say',
 'work',
 'thought',
 'bit',
 'found',
 'year',
 'going',
 'give',
 'interesting',
 'liked',
 'writing',
 'novel',
 'loved',
 'another',
 'feel',
 'better',
 'got',
 'come',
 'man',
 'hot',
 'still',
 'enough',
 'back',
 'though',
 'people',
 'star',
 'reader',
 'made',
 'review',
 'something',
 'part',
 'friend',
 'cant',
 'page',
 'need',
 'bad',
 'keep',
 'new',
 'wasnt',
 'free',
 'world',
 'doesnt',
 'relationship',
 'enjoy',
 'recommend',
 'together',
 'next',
 'start',
 'felt',
 'best',
 'put',

In [ ]:
def Avg_Word2Vec(doc):
  words_in_vocab = [word for word in doc if word in model.wv.index_to_key]
  if not words_in_vocab:
    return np.zeros(model.vector_size)
  return np.mean([model.wv[word] for word in words_in_vocab], axis=0)

In [ ]:
from tqdm import tqdm

In [ ]:
X = []
for i in tqdm(range(len(words))):
  X.append(Avg_Word2Vec(words[i]))

100%|██████████| 12000/12000 [00:15<00:00, 798.61it/s]


In [ ]:
X_new = np.array(X)

In [ ]:
df

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read i didnt want put i read one s...,1
2,ill start saying first four book i wasnt expec...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,i expect type book library pleased find price ...,1
...,...,...
11995,valentine cupid vampire jena ian another vampi...,1
11996,i read seven book series apocalypticadventure ...,1
11997,this book really wasnt cuppa the situation man...,1
11998,tried use charge kindle didnt even register ch...,0


In [ ]:
y = df['rating']

In [ ]:
ndf = pd.DataFrame()
X_list = []
for i in range(0, len(X)):
  X_list.append(pd.DataFrame(X[i].reshape(1, -1)))

ndf = pd.concat(X_list, ignore_index=True)

In [ ]:
ndf.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.036321,0.338075,0.191755,0.127381,0.015788,-0.473477,0.076451,0.761450,-0.099906,-0.156054,...,0.443995,0.075846,-0.127888,-0.096561,0.452806,0.148706,0.093542,-0.340897,0.162472,-0.035928
1,-0.287065,0.299929,0.071389,0.384865,0.261241,-0.537448,0.330418,0.886643,-0.362591,-0.395250,...,0.824192,0.228173,-0.115926,0.028413,0.458418,0.352943,0.059014,-0.263575,0.258068,-0.344425
2,-0.238801,0.332400,0.244846,0.176120,0.169349,-0.548041,0.322985,0.750160,-0.201219,-0.356142,...,0.660912,0.107093,-0.016668,-0.056041,0.476193,0.386687,0.079962,-0.202399,0.150998,-0.104170
3,-0.323876,0.305303,0.224374,0.360253,0.164559,-0.552983,0.333441,0.715201,-0.131528,-0.317521,...,0.536509,0.130138,-0.005665,-0.095977,0.339635,0.435556,0.222272,0.099500,0.080509,-0.256043
4,-0.029433,0.338978,-0.038299,-0.118944,0.013775,-0.477998,0.094150,0.770433,-0.230802,-0.416495,...,0.646650,0.184075,0.348116,-0.045261,0.546384,0.356862,0.143925,-0.393440,0.216643,-0.106655


In [ ]:
ndf['Output'] = y

In [ ]:
X=df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ndf.drop('Output', axis=1), y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [ ]:
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.7695833333333333

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.58      0.62       782
           1       0.81      0.86      0.83      1618

    accuracy                           0.77      2400
   macro avg       0.74      0.72      0.73      2400
weighted avg       0.76      0.77      0.76      2400

